In [43]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from tqdm import tqdm
import shap

# Load Data

In [44]:
# classifiers = pd.read_csv('classifiers.csv', sep=';')
subjects = pd.read_csv('data/subjects.csv', sep=';', encoding='windows-1251')

diagnosticresults = pd.read_csv('data_val/diagnosticresults.csv', sep=';').drop(['Partition'], axis=1)
diagnosticresultdetails = pd.read_csv('data_val/diagnosticresultdetails.csv', sep=';').drop(['Partition'], axis=1)

egeresults = pd.read_csv('data_val/egeresults.csv', sep=';').drop(['Partition'], axis=1)

medalistresults = pd.read_csv('data_val/medalistresults.csv', sep=';').drop(['Partition'], axis=1)
medalistsubjects = pd.read_csv('data_val/medalistsubjects.csv', sep=';').drop(['Partition'], axis=1)

participants = pd.read_csv('data_val/participants.csv', sep=';').drop(['Partition'], axis=1)

selfdiagnosticresults = pd.read_csv('data_val/selfdiagnosticresults.csv', sep=';').drop(['Partition'], axis=1)
selfdiagnosticresultdetails = pd.read_csv('data_val/selfdiagnosticresultdetails.csv', sep=';').drop(['Partition'], axis=1)

# Feature Generation

In [46]:
participants = participants.rename(columns={'Id': 'ParticipantId'})

In [47]:
diagnosticresults['MarkPercent'] = round(100 * diagnosticresults.Mark / diagnosticresults.MaxMark)
selfdiagnosticresults['MarkPercent'] = round(100 * selfdiagnosticresults.Mark / selfdiagnosticresults.MaxMark)

In [48]:
diagnosticresults_mean_mark = pd.DataFrame(diagnosticresults.groupby(['ParticipantId', 'SubjectId']).MarkPercent.mean()).reset_index()
selfdiagnosticresults_mean_mark = pd.DataFrame(selfdiagnosticresults.groupby(['ParticipantId', 'SubjectId']).MarkPercent.mean()).reset_index()

In [49]:
data = pd.merge(egeresults, participants, on=['ParticipantId'])

In [50]:
data.head()

,Id,ParticipantId,DateExam,MarkPercent,SubjectId,Parallel,OOCode
0,00262026-7B36-48BF-859D-36D55998EFCB,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-10,87,49,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9
1,13CE9B68-57E9-4888-9F05-0BB229A70DD1,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-13,69,77,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9
2,ADDA517A-C043-4B25-AA59-0E07BC0101E7,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-03,85,27,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9
3,FE773762-5D87-446A-AB7A-025AED917853,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-06,86,75,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9
4,00284E0B-A829-4244-A877-26607175F292,E8B2DBBD-A7D3-4ABA-82A4-FB8E3DBC3194,2020-07-10,46,49,11,9B6BA432-DCE2-4ECF-9A86-2B9350634A17


In [51]:
subjects_compliance = dict()
for id in subjects[subjects.Id.isin(egeresults.SubjectId.unique())].Id.values:
    subjects_compliance[id] = []
    subjects_compliance[id].append(id)

subjects_compliance[4].append(5)
subjects_compliance[5].append(4)

subjects_compliance[49].append(3)
subjects_compliance[49].append(13)
subjects_compliance[49].append(48)

subjects_compliance[31].append(11)
subjects_compliance[31].append(32)

subjects_compliance[61].append(91)

In [52]:
diagnosticresults_processed = pd.DataFrame()
for key in subjects_compliance:
    temp = pd.DataFrame(diagnosticresults[diagnosticresults.SubjectId.isin(subjects_compliance[key])] \
                                         .groupby('ParticipantId').MarkPercent.min()).reset_index() \
                                         .rename(columns={'MarkPercent': 'DiagnosticMarkPercentMin'})
    temp['DiagnosticMarkPercentMean'] = pd.DataFrame(diagnosticresults[diagnosticresults.SubjectId.isin(subjects_compliance[key])] \
                                                                      .groupby('ParticipantId').MarkPercent.mean()).reset_index() \
                                                                      .rename(columns={'MarkPercent': 'DiagnosticMarkPercentMean'})['DiagnosticMarkPercentMean']
    temp['DiagnosticMarkPercentMax'] = pd.DataFrame(diagnosticresults[diagnosticresults.SubjectId.isin(subjects_compliance[key])] \
                                                                      .groupby('ParticipantId').MarkPercent.max()).reset_index() \
                                                                      .rename(columns={'MarkPercent': 'DiagnosticMarkPercentMax'})['DiagnosticMarkPercentMax']
    temp['SubjectId'] = key
    diagnosticresults_processed = pd.concat([diagnosticresults_processed,
                                            temp])
    
data = pd.merge(data, diagnosticresults_processed, on=['ParticipantId', 'SubjectId'], how='left')

In [53]:
diagnosticresults_processed

,ParticipantId,DiagnosticMarkPercentMin,DiagnosticMarkPercentMean,DiagnosticMarkPercentMax,SubjectId
0,0212F5D3-969B-40DB-B0EB-20AB2CCC8F28,50.0,50.0,50.0,4
1,04266577-CEB3-410B-A594-0A607880ED9C,91.0,91.0,91.0,4
2,04394E73-1B90-4FF2-AFE5-46D95D5B4DA6,86.0,86.0,86.0,4
3,084FB3F4-A52E-48B1-B687-61287B8D4E13,86.0,86.0,86.0,4
4,0C839F26-56BD-4D0D-B051-E02176E69D54,69.0,69.0,69.0,4
...,...,...,...,...,...
8,9B0B1921-D6B1-4778-A5BD-3CD0767D0198,42.0,42.0,42.0,86
9,B367FF12-7C71-4EB6-9C73-7CD4275A332A,87.0,87.0,87.0,86
10,E8001BA2-7D46-421A-AA24-F199B5867992,53.0,54.5,56.0,86
11,EC0DE436-628C-4373-9847-B372C39A8B48,35.0,35.0,35.0,86


In [54]:
selfdiagnosticresults_processed = pd.DataFrame()
for key in subjects_compliance:
    temp = pd.DataFrame(selfdiagnosticresults[selfdiagnosticresults.SubjectId.isin(subjects_compliance[key])] \
                                         .groupby('ParticipantId').MarkPercent.min()).reset_index() \
                                         .rename(columns={'MarkPercent': 'SelfDiagnosticMarkPercentMin'})
    temp['SelfDiagnosticMarkPercentMean'] = pd.DataFrame(selfdiagnosticresults[selfdiagnosticresults.SubjectId.isin(subjects_compliance[key])] \
                                                                      .groupby('ParticipantId').MarkPercent.mean()).reset_index() \
                                                                      .rename(columns={'MarkPercent': 'DiagnosticMarkPercentMean'})['DiagnosticMarkPercentMean']
    temp['SelfDiagnosticMarkPercentMax'] = pd.DataFrame(selfdiagnosticresults[selfdiagnosticresults.SubjectId.isin(subjects_compliance[key])] \
                                                                      .groupby('ParticipantId').MarkPercent.max()).reset_index() \
                                                                      .rename(columns={'MarkPercent': 'DiagnosticMarkPercentMax'})['DiagnosticMarkPercentMax']
    temp['SubjectId'] = key
    selfdiagnosticresults_processed = pd.concat([selfdiagnosticresults_processed,
                                                 temp])
    
data = pd.merge(data, selfdiagnosticresults_processed, on=['ParticipantId', 'SubjectId'], how='left')

In [55]:
selfdiagnosticresults_processed

,ParticipantId,SelfDiagnosticMarkPercentMin,SelfDiagnosticMarkPercentMean,SelfDiagnosticMarkPercentMax,SubjectId
0,0C839F26-56BD-4D0D-B051-E02176E69D54,0.0,0.000000,0.0,4
1,11A37026-6A8D-449D-9CA5-8A8D58B38094,18.0,45.333333,74.0,4
2,13B39F59-DAF7-449A-A21A-2A6A38D58FA5,6.0,50.000000,100.0,4
3,14235D5E-484D-40D0-8BF1-5293E0E79C19,0.0,48.555556,89.0,4
4,199FD41F-333A-4648-A1F7-FBE8927A0358,0.0,16.500000,33.0,4
...,...,...,...,...,...
23,B9CBE51A-E237-4D0F-AEEF-BE84AF148AA1,38.0,54.300000,63.0,86
24,C84F3D0A-F092-4A3D-A315-96DE8DAD4FD4,96.0,98.000000,100.0,86
25,CF620915-5C3C-495A-95EE-EAEB49378971,0.0,25.250000,92.0,86
26,D6837150-C6A3-469F-A016-1827050D02C4,5.0,66.000000,93.0,86


In [56]:
diagnosticresults_processed_all = pd.DataFrame(diagnosticresults.groupby('ParticipantId').MarkPercent.min()) \
                                     .reset_index().rename(columns={'MarkPercent':'AllDiagnosticMarkPercentMin'})
diagnosticresults_processed_all['AllDiagnosticMarkPercentMean'] = diagnosticresults.groupby('ParticipantId').MarkPercent.mean().values
diagnosticresults_processed_all['AllDiagnosticMarkPercentMax'] = diagnosticresults.groupby('ParticipantId').MarkPercent.max().values

data = pd.merge(data, diagnosticresults_processed_all, on=['ParticipantId'], how='left')

In [57]:
diagnosticresults_processed_all

,ParticipantId,AllDiagnosticMarkPercentMin,AllDiagnosticMarkPercentMean,AllDiagnosticMarkPercentMax
0,0028C3A9-740B-4639-B569-73E068A69508,75.0,75.0,75.0
1,003B37F1-E178-4A82-A4D3-8F7151E2E572,15.0,15.0,15.0
2,004F34A9-67D5-420F-8A83-067B7EF91A78,45.0,45.0,45.0
3,005827FF-3C8C-46CE-8EAD-A8C3BD26AADC,10.0,10.0,10.0
4,008B2B5F-10EA-45BB-B5BC-3937619842E8,25.0,25.0,25.0
...,...,...,...,...
1233,FED441E7-CBB0-46E5-9D81-4519229C1954,60.0,60.0,60.0
1234,FED4817F-555C-4615-A907-41AB1DE3D054,30.0,30.0,30.0
1235,FEE73DC7-27BD-44CB-9D3A-5E8D8721C9EF,0.0,0.0,0.0
1236,FF07EEE0-2A4C-4911-BD8B-16B5B9E85E6C,45.0,45.0,45.0


In [58]:
selfdiagnosticresults_processed_all = pd.DataFrame(selfdiagnosticresults.groupby('ParticipantId').MarkPercent.min()) \
                                     .reset_index().rename(columns={'MarkPercent':'AllSelfDiagnosticMarkPercentMin'})
selfdiagnosticresults_processed_all['AllSelfDiagnosticMarkPercentMean'] = selfdiagnosticresults.groupby('ParticipantId').MarkPercent.mean().values
selfdiagnosticresults_processed_all['AllSelfDiagnosticMarkPercentMax'] = selfdiagnosticresults.groupby('ParticipantId').MarkPercent.max().values

data = pd.merge(data, selfdiagnosticresults_processed_all, on=['ParticipantId'], how='left')

In [59]:
selfdiagnosticresults_processed_all

,ParticipantId,AllSelfDiagnosticMarkPercentMin,AllSelfDiagnosticMarkPercentMean,AllSelfDiagnosticMarkPercentMax
0,0024D81B-E86A-4B78-8835-05268B3AE4A2,0.0,58.250000,97.0
1,02F93843-0DBC-45E9-8480-3E9DCF3FE9AB,6.0,43.750000,81.0
2,0359CD1B-C97D-4F30-BABB-9A88C20EC1F8,38.0,47.000000,56.0
3,04F526CF-E495-4762-8918-18D64F8AE03D,34.0,48.750000,70.0
4,05C275FB-49EE-46E8-AE3A-054E97725FB8,33.0,65.666667,84.0
...,...,...,...,...
299,FA588313-6917-44C9-9054-038B6E809896,89.0,94.333333,100.0
300,FBA4367E-4664-4D32-9494-32A30E4B0D9A,16.0,43.333333,67.0
301,FCDD3EE7-8BB6-4A16-A05B-063E273A1D6B,40.0,40.000000,40.0
302,FF78C79C-4FC3-47C7-8737-4680875E96C8,2.0,43.333333,94.0


In [60]:
data.head()

,Id,ParticipantId,DateExam,MarkPercent,SubjectId,Parallel,OOCode,DiagnosticMarkPercentMin,DiagnosticMarkPercentMean,DiagnosticMarkPercentMax,SelfDiagnosticMarkPercentMin,SelfDiagnosticMarkPercentMean,SelfDiagnosticMarkPercentMax,AllDiagnosticMarkPercentMin,AllDiagnosticMarkPercentMean,AllDiagnosticMarkPercentMax,AllSelfDiagnosticMarkPercentMin,AllSelfDiagnosticMarkPercentMean,AllSelfDiagnosticMarkPercentMax
0,00262026-7B36-48BF-859D-36D55998EFCB,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-10,87,49,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.0,45.0,NaN,NaN,NaN
1,13CE9B68-57E9-4888-9F05-0BB229A70DD1,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-13,69,77,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.0,45.0,NaN,NaN,NaN
2,ADDA517A-C043-4B25-AA59-0E07BC0101E7,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-03,85,27,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.0,45.0,NaN,NaN,NaN
3,FE773762-5D87-446A-AB7A-025AED917853,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-06,86,75,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9,NaN,NaN,NaN,NaN,NaN,NaN,45.0,45.0,45.0,NaN,NaN,NaN
4,00284E0B-A829-4244-A877-26607175F292,E8B2DBBD-A7D3-4ABA-82A4-FB8E3DBC3194,2020-07-10,46,49,11,9B6BA432-DCE2-4ECF-9A86-2B9350634A17,NaN,NaN,NaN,0.0,29.555556,41.0,10.0,35.0,60.0,0.0,29.608696,47.0


In [61]:
data['IsMedalist'] = 0
data.loc[data.ParticipantId.isin(medalistresults[medalistresults.Type == 1].ParticipantId.values), 'IsMedalist'] = 1

In [62]:
medalistsubjects = medalistsubjects.drop(['Id'], axis=1).rename(columns={'ResultId': 'Id'})

In [63]:
temp = pd.merge(medalistresults[medalistresults.Type == 0], medalistsubjects, on='Id', how='left')

In [64]:
temp.groupby('ParticipantId').Type.count().sort_values()

ParticipantId
24CD1639-AD51-42E1-A552-2AC2EDC3D9DC    1
76075DCE-36AE-4137-B940-8897AFA50298    1
786C12B4-152E-43AE-B0EC-7D210C9FA94D    1
F269171A-23BF-4D9A-B8B2-B10599DBF55F    1
FA4EC66E-72A2-433A-8BE0-0EC102F22BBE    3
Name: Type, dtype: int64

In [65]:
temp[temp.ParticipantId == 'E8B0231E-A8D2-4F0B-882B-7DB5EAFE9D76']

,Id,Type,ParticipantId,SubjectId


In [66]:
temp = temp[['ParticipantId', 'SubjectId']].copy()
temp['IsOlimpiad'] = 1

In [67]:
data = pd.merge(data, temp, on=['ParticipantId', 'SubjectId'], how='left')
data.loc[data.IsOlimpiad.isnull(), 'IsOlimpiad'] = 0
data.IsOlimpiad = data.IsOlimpiad.astype(int)

In [68]:
data.head(1)

,Id,ParticipantId,DateExam,MarkPercent,SubjectId,Parallel,OOCode,DiagnosticMarkPercentMin,DiagnosticMarkPercentMean,DiagnosticMarkPercentMax,...,SelfDiagnosticMarkPercentMean,SelfDiagnosticMarkPercentMax,AllDiagnosticMarkPercentMin,AllDiagnosticMarkPercentMean,AllDiagnosticMarkPercentMax,AllSelfDiagnosticMarkPercentMin,AllSelfDiagnosticMarkPercentMean,AllSelfDiagnosticMarkPercentMax,IsMedalist,IsOlimpiad
0,00262026-7B36-48BF-859D-36D55998EFCB,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-10,87,49,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9,NaN,NaN,NaN,...,NaN,NaN,45.0,45.0,45.0,NaN,NaN,NaN,0,0


In [69]:
pd.DataFrame([[idx, sub, mark_changes]])

,0,1,2
0,5581C621-5314-4B07-8D32-1B0357AE30CE,51,0


In [70]:
diagnosticresults_dynamic = pd.DataFrame(columns=['ParticipantId', 'SubjectId', 'DiagnosticMarkPercentDynamic'])

for idx in tqdm(diagnosticresults.ParticipantId.unique()):
    temp_dynamic = pd.DataFrame()
    for sub in diagnosticresults[diagnosticresults.ParticipantId == idx].SubjectId.unique():
        temp = diagnosticresults[(diagnosticresults.ParticipantId == idx)&(diagnosticresults.SubjectId == sub)] \
                                .set_index(['ParticipantId', 'SubjectId']).sort_values('TestDate')
        mark_changes = 0
        if len(temp.MarkPercent.values) < 2:
            pass
        else:
            marks = temp.MarkPercent.values
            for i in range(len(temp.MarkPercent.values)-1):
                mark_changes += marks[i+1] - marks[i]
            mark_changes /= len(marks)-1
#         print(sub, mark_changes)
        temp_dynamic = pd.concat([temp_dynamic, 
                                  pd.DataFrame([[idx, sub, mark_changes]],
                                               columns=['ParticipantId', 'SubjectId', 'DiagnosticMarkPercentDynamic'])])
    diagnosticresults_dynamic = pd.concat([diagnosticresults_dynamic, temp_dynamic])
#     break

100%|█████████████████████████████████████████████████████████████████████████████| 1238/1238 [00:08<00:00, 140.45it/s]


In [71]:
diagnosticresults_dynamic

,ParticipantId,SubjectId,DiagnosticMarkPercentDynamic
0,04266577-CEB3-410B-A594-0A607880ED9C,75,0.00
0,04266577-CEB3-410B-A594-0A607880ED9C,48,0.00
0,04266577-CEB3-410B-A594-0A607880ED9C,4,0.00
0,04266577-CEB3-410B-A594-0A607880ED9C,68,0.00
0,2AF4F7CB-5A44-42CC-B41D-4F72299A6A1A,31,-0.25
...,...,...,...
0,46A5EDCE-DAF9-47F9-828B-AFC6FF0140EE,36,0.00
0,B521A760-288C-4B5A-AFE8-DB563CFAFEB7,36,0.00
0,A094D698-DC76-4D74-AA60-21B43DE16417,51,0.00
0,B45B387C-619E-4DDB-8156-939E839DD9DB,51,0.00


In [72]:
data = pd.merge(data, diagnosticresults_dynamic, on=['ParticipantId', 'SubjectId'], how='left')

In [73]:
data.head()

,Id,ParticipantId,DateExam,MarkPercent,SubjectId,Parallel,OOCode,DiagnosticMarkPercentMin,DiagnosticMarkPercentMean,DiagnosticMarkPercentMax,...,SelfDiagnosticMarkPercentMax,AllDiagnosticMarkPercentMin,AllDiagnosticMarkPercentMean,AllDiagnosticMarkPercentMax,AllSelfDiagnosticMarkPercentMin,AllSelfDiagnosticMarkPercentMean,AllSelfDiagnosticMarkPercentMax,IsMedalist,IsOlimpiad,DiagnosticMarkPercentDynamic
0,00262026-7B36-48BF-859D-36D55998EFCB,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-10,87,49,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9,NaN,NaN,NaN,...,NaN,45.0,45.0,45.0,NaN,NaN,NaN,0,0,NaN
1,13CE9B68-57E9-4888-9F05-0BB229A70DD1,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-13,69,77,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9,NaN,NaN,NaN,...,NaN,45.0,45.0,45.0,NaN,NaN,NaN,0,0,NaN
2,ADDA517A-C043-4B25-AA59-0E07BC0101E7,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-03,85,27,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9,NaN,NaN,NaN,...,NaN,45.0,45.0,45.0,NaN,NaN,NaN,0,0,NaN
3,FE773762-5D87-446A-AB7A-025AED917853,FB18BA15-0699-47CD-8346-DDA81643CD39,2020-07-06,86,75,11,B8FE2712-B51A-4D47-B0E9-B679BA4CC5E9,NaN,NaN,NaN,...,NaN,45.0,45.0,45.0,NaN,NaN,NaN,0,0,NaN
4,00284E0B-A829-4244-A877-26607175F292,E8B2DBBD-A7D3-4ABA-82A4-FB8E3DBC3194,2020-07-10,46,49,11,9B6BA432-DCE2-4ECF-9A86-2B9350634A17,NaN,NaN,NaN,...,41.0,10.0,35.0,60.0,0.0,29.608696,47.0,0,0,NaN


# Test model

In [74]:
test = data.drop(['Id', 'ParticipantId', 'DateExam', 'OOCode'], axis=1)

In [75]:
X = test.drop(['MarkPercent'], axis=1)
y = test['MarkPercent']

In [76]:
models = []
for i in range(1, 6):
    model = CatBoostRegressor()
    model.load_model('models/model_{}'.format(i))
    models.append(model)

In [88]:
prediction = np.zeros((X.shape[0]))

for i in range(5):
    prediction += models[i].predict(X) / 5

In [89]:
prediction

array([43.90785395, 60.10512811, 61.46236157, ..., 23.94172498,
       69.30292772, 33.11155478])

In [90]:
mean_squared_error(y, 
                   prediction, 
                   squared=False)

15.687718571570054

In [92]:
mean_squared_error(y[X.DiagnosticMarkPercentMin.notnull()], 
                   prediction[X.DiagnosticMarkPercentMin.notnull()], 
                   squared=False)

12.727826309423742